In [2]:
from sklearn.cluster import KMeans
from transformers import BertTokenizer, BertModel
import torch
import re
import pandas as pd
from datetime import datetime

def getRightNow():
    now = datetime.now()
    hms = now.strftime("%H:%M:%S")
    return hms
print("Imports {}".format(getRightNow()))

Imports 08:25:34


In [5]:
outfile = "output_file.txt"
infile = "Data_June_16_2023_salsify.xlsx"
def ETL():
    def convert_numbers_to_x(string):
        # Convert "blah blah size 6" or "blah blah size 16" to "blah blah size x"
        return re.sub(r'\d+', 'x', string)

    data = pd.read_excel(infile)
    # Remove quotes, special characters, and backslashes from cells
    data = data.applymap(lambda x: str(x).replace('"', '').replace('|', '').replace('\n', '').replace('\\', '').replace('"','').replace('™', '').replace('®', '').replace(' -','')) 
    # Drop rows with duplicate values in the last column
    data.drop_duplicates(subset=data.columns[-1], keep=False, inplace=True)
    # Convert numbers to 'x'
    data = data.applymap(convert_numbers_to_x)
    # Convert DataFrame to PIPE delimited flat file
    output = data.apply(lambda x: '|'.join(x.astype(str)), axis=1)
    # Write the output to a file
    outfile = "output_file.txt"
    output.to_csv(outfile, index=False, header=False)
    # print first 10 things
    print(output[0:10])
print("{} => {} @ {}".format( infile, outfile, getRightNow()))

Data_June_16_2023_salsify.xlsx => output_file.txt @ 08:30:41


In [8]:
# List of product descriptions
product_descriptions = [
    "High-quality leather wallet for men",
    "Elegant and stylish women's handbag",
    "Durable and lightweight hiking backpack",
    "Soft and comfortable cotton t-shirt",
    "Premium stainless steel kitchen knife set"
]

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode the product descriptions
encoded_inputs = tokenizer(product_descriptions, padding=True, truncation=True, return_tensors='pt')

# Get the embeddings using BERT model
with torch.no_grad():
    outputs = model(**encoded_inputs)

# Get the sentence-level embeddings (CLS token)
embeddings = outputs.last_hidden_state[:, 0, :]

# Convert embeddings to numpy array
embeddings = embeddings.numpy()

# Perform clustering using K-Means algorithm
num_clusters = 2  # Number of clusters to create
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
cluster_labels = kmeans.fit_predict(embeddings)

# Print the cluster labels for each product description
for i, desc in enumerate(product_descriptions):
    print(f"Product: {desc}\tCluster: {cluster_labels[i]}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Product: High-quality leather wallet for men	Cluster: 1
Product: Elegant and stylish women's handbag	Cluster: 0
Product: Durable and lightweight hiking backpack	Cluster: 1
Product: Soft and comfortable cotton t-shirt	Cluster: 0
Product: Premium stainless steel kitchen knife set	Cluster: 0
